Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos
<br>

<center>
    <h2> Tarea 5 </h2>
    <h1> SVM </h1>
    <p>
        Profesor Marcelo Mendoza<br>
        Segundo Semestre 2023<br> 
        Fecha de entrega: 3 de noviembre
    </p>
    <br>
</center>

<br>


## Indicaciones

Deberás entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas.

**IMPORTANTE**:
- Se te dará puntaje tanto por código como por la manera en la que respondas las preguntas planteadas. Es decir, si tienes un código perfecto pero este no es explicado o no se responden preguntas asociadas a este, no se tendrá el puntaje completo.
- El notebook debe tener todas las celdas de código ejecutadas. Cualquier notebook que no las tenga no podrá ser corregido.
- El carácter de esta tarea es **INDIVIDUAL**. Cualquier instancia de copia resultará en un 1,1 como nota de curso.
- En el caso de que se encuentren con problemas al correr celdas por el tamaño del dataset, esta permitido trabajar con una muestra representativa de este, siempre explicitando y justificando sus deciciones.

---


## Librerías

In [1]:
##Importa acá las librerias que vayas a utilizar
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from lime.lime_text import LimeTextExplainer

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Contexto:

Se tiene un dataset con información de distintas noticias, entre las cuales hay noticias falsas y verdaderas. El objetivo de esta tarea es predecir si una noticia es falsa o verdadera, utilizando distintos modelos de clasificación.. Para esto, primero se debe hacer un análisis exploratorio de los datos para decidir qué variables son relevantes para el problema. Luego, se debe vectorizar el texto de loas noticias para poder utilizarlo en un modelo de clasificación (en esta tarea usaremos SBert y TF-IDF). Después se debe entrenar un modelo de clasificación SVM que prediga si una noticia es falsa o verdadera. Finalmente, se debe evaluar el modelo y compararlo con otros modelos de clasificación.

# Objetivos de la tarea:

- Realizar un análisis exploratorio de datos para determinar las variables pertinentes para el problema en cuestión, identificando características clave y patrones en los datos que podrían afectar la clasificación de las noticias.
- Dominar la técnica de vectorización de texto para facilitar su aplicación en un modelo de clasificación, y comprender la interpretación de los resultados obtenidos.
- Entender el funcionamiento de un modelo de clasificación SVM y sus hiperparámetros.
- Entrenar un modelo de clasificación SVM capaz de predecir si una noticia es falsa o verdadera.
- Evaluar el desempeño del modelo y analizar en profundidad los resultados obtenidos.

# Parte 1: Carga y Preprocesamiento (10 puntos)

## 1.1 Carga de datos (1 punto)

Para esta tarea deberás trabajar con el dataset que está en Canvas, 'news.csv'. Este dataset contiene información de distintas noticias, entre las cuales hay noticias falsas y verdaderas.

## 1.2 Descripcion del Dataset (5 puntos)



¿Qué representa cada feature en el dataset entregado? Refiérete a su tipo de dato, detallando cómo trabajar con los datos no númericos (2 puntos)

RESPUESTA:


¿Qué columnas o features crees son relevantes para el problema? ¿Por qué? (3 puntos)

RESPUESTA:

## 1.3 Datos nulos (2 puntos)

Analiza la presencia de valores nulos en el conjunto de datos y cómo se distribuyen en las distintas columnas. Después, toma una decisión acerca del tratamiento adecuado para el dataframe con respecto a los valores nulos. Justifica tu decisión.

RESPUESTA:


## 1.4 Manejo del Dataset (2 puntos)

- Elimina las columnas que no sean relevantes para el entrenamiento del modelo (1 pts.)
- Haz que los textos estén en un formato óptimo para ser procesados por los modelos de clasificación (0.5 pts.) Justifica tu decisión (0.5 pts.)



RESPUESTA:

# Parte 2: Vectorización (14 puntos)

# SBert

Para esta parte, deben utilizar SBert para vectorizar los textos de las noticias. Utilicen el modelo pre-entrenado de SBert llamado SentenceTransformer. Específicamente, deben usar el modelo 'paraphrase-MiniLM-L6-v2' para obtener las representaciones vectoriales de las oraciones.

### 2.1) Vectorización con SBert (3 puntos)

### 2.2) Análisis teórico de SBert (4 puntos)
Responde las siguientes preguntas: ¿Qué es SBert? ¿Cómo funciona? ¿Qué ventajas y desventajas tiene sobre otros modelos de vectorización de texto?

RESPUESTA:

# TF-IDF

Para esta parte, deben utilizar TF-IDF para vectorizar los textos de las noticias. Utilicen la clase TfidfVectorizer de la librería sklearn.


Tip: limita la cantidad de features.

### 2.3) Vectorización con TF-IDF (3 puntos)

### 2.2) Análisis teórico de TF-IDF (4 puntos)
Responde las siguientes preguntas: ¿Qué es TF-IDF? ¿Cómo funciona? ¿Qué ventajas y desventajas tiene sobre otros modelos de vectorización de texto?

RESPUESTA:

# Parte 3: SVM (36 puntos)

### 3.1) Preguntas teóricas (10 puntos)
1. ¿Qué es SVM? ¿Cómo funciona? ¿En qué casos es útil? ¿En cuáles no? (4 ptos.)
3. ¿Qué es un kernel? ¿Qué tipos de kernels existen? ¿Cuál es la diferencia entre ellos? (3 ptos.)
4. ¿Qué indica el parámetro C? ¿Qué sucede si C es muy grande? ¿Y si es muy pequeño? (3 ptos.)

### 3.2) SVM con vectorización SBert (10 puntos) 
Para esta parte deben entrenar un modelo SVM con las representaciones vectoriales obtenidas con SBert. Deben utilizar la clase SVC de la librería sklearn. Para esto:
- Deben dejar los datos en un formato adecuado para ser procesados por el modelo (1 pto.)
- Dividir el dataset en train y test (1 pto.)
- Entrenar el modelo SVM con los datos de train (2 ptos.) modificando los hiperparámetros kernel y C (4 ptos.)
- Evaluar el modelo con los datos de test y comentar brevemente los resultados obtenidos (2 ptos.).

### 3.3) SVM con vectorización TF-IDF (10 puntos) 
Para esta parte deben entrenar un modelo SVM con las representaciones vectoriales obtenidas con TF-IDF. Deben utilizar la clase SVC de la librería sklearn. Para esto:
- Deben dejar los datos en un formato adecuado para ser procesados por el modelo (1 pto.)
- Dividir el dataset en train y test (1 pto.)
- Entrenar el modelo SVM con los datos de train (2 ptos.) modificando los hiperparámetros kernel y C (4 ptos.)
- Evaluar el modelo con los datos de test y comentar brevemente los resultados obtenidos (2 ptos.).

### 3.4) Análisis de resultados (6 puntos)
- ¿Qué vectorización obtuvo mejores resultados, SBert o TF-IDF? ¿Por qué? (3 ptos.)
- ¿Qué hiperparámetros obtuvieron mejores resultados para cada vectorización? (3 ptos.)

# Parte 4 (Bonus): LIME explainer (5 puntos)

A continuación haz un análisis de los resultados obtenidos con el modelo SVM con vectorización SBert utilizando LIME explainer. Para esto, debes seguir los siguientes pasos:
- Instalar la librería Lime
- Elegir un ejemplo de test
- Utilizar el explainer de Lime para explicar la predicción del modelo en ese ejemplo (2.5 pts.)
- Analizar los resultados obtenidos y comentar brevemente (2.5 pts.)